In [5]:
import tensorflow as tf

# 介绍
mnist是一个数字图像的数据集，每个图像都是28*28的。这里，我们将做一个分类算法，识别图像中的数字（0-9）。

这里，我们将对上节课的模型进行修改，使用CNN进一步提高准确率。同时，还将添加一些功能。

In [6]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


# 模型
这里我们使用的模型是 y = softmax(Wx + b)。

具体的模型设计是机器学习原理的问题，这里我们只将实现。具体的计算非常简单，分为三步：
1. 建立模型
2. 建立loss计算公式
3. 选择梯度下降算法

CNN模型：
- conv[5,5], 32 + relu + max-pooling
- conv[5,5], 64 + relu + max-pooling
- FC:64



In [7]:
x = tf.placeholder(tf.float32, [None, 784]) 
y = tf.placeholder(tf.float32, [None, 10])


In [8]:
def conv2d(x, W):
    """conv2d returns a 2d convolution layer with full stride."""
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def max_pool_2x2(x):
    """max_pool_2x2 downsamples a feature map by 2X."""
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')


def weight_variable(shape):
    """weight_variable generates a weight variable of a given shape."""
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    """bias_variable generates a bias variable of a given shape."""
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

with tf.name_scope('reshape'):
    x_image = tf.reshape(x, [-1, 28, 28, 1])

# First convolutional layer - maps one grayscale image to 32 feature maps.
with tf.name_scope('conv1'):
    W_conv1 = weight_variable([5, 5, 1, 32])
    b_conv1 = bias_variable([32])
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)

# Pooling layer - downsamples by 2X.
with tf.name_scope('pool1'):
    h_pool1 = max_pool_2x2(h_conv1)

# Second convolutional layer -- maps 32 feature maps to 64.
with tf.name_scope('conv2'):
    W_conv2 = weight_variable([5, 5, 32, 64])
    b_conv2 = bias_variable([64])
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)

# Second pooling layer.
with tf.name_scope('pool2'):
    h_pool2 = max_pool_2x2(h_conv2)

# Fully connected layer 1 -- after 2 round of downsampling, our 28x28 image
# is down to 7x7x64 feature maps -- maps this to 1024 features.
with tf.name_scope('fc1'):
    W_fc1 = weight_variable([7 * 7 * 64, 1024])
    b_fc1 = bias_variable([1024])

    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# Dropout - controls the complexity of the model, prevents co-adaptation of
# features.
with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# Map the 1024 features to 10 classes, one for each digit
with tf.name_scope('fc2'):
    W_fc2 = weight_variable([1024, 10])
    b_fc2 = bias_variable([10])

    logits = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

    y_ = tf.nn.softmax(logits)


In [9]:
#loss
loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y)

In [10]:
#梯度下降
optimizer = tf.train.AdamOptimizer(1e-4)
train = optimizer.minimize(loss)

# 运行

In [11]:
# Interactive Session 可以保证你运行的所有代码都在sess中？？？
sess = tf.InteractiveSession()

In [12]:
# 记住在使用变量前必须要进行初始化
tf.global_variables_initializer().run()

In [13]:
# 使用mnist数据训练1000个epoch（回合）
epoch = 1000
for _ in range(epoch):
    x_batch, y_batch = mnist.train.next_batch(100)
    sess.run(train, feed_dict = {x: x_batch, y: y_batch, keep_prob: 0.5})

# 测试

In [14]:
# 首先我们建立一个计算accuracy的公式,原理就是比较预测和实际值然后求平均值
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

#同样的，放到sess中就可以计算
sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob: 1.0})

0.97060001